<a href="https://colab.research.google.com/github/pkdart13/Colab-Hacks/blob/master/HACKANONS_COLAB_25GB_RAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title **Create User**
#@markdown Enter Username and Password

import os

username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}

print("Creating User and Setting it up")

# Creation of user
os.system(f"useradd -m {username}")

# Add user to sudo group
os.system(f"adduser {username} sudo")
    
# Set password of user to 'root'
os.system(f"echo '{username}:{password}' | sudo chpasswd")

# Change default shell from sh to bash
os.system("sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd")

print("User Created and Configured")

Creating User and Setting it up
User Created and Configured


In [2]:
#@title **RDP**
#@markdown  It takes 4-5 minutes for installation

import os
import subprocess

#@markdown  Visit http://remotedesktop.google.com/headless and Copy the command after authentication

CRP = "DISPLAY= /opt/google/chrome-remote-desktop/start-host --code=\"4/0AX4XfWhZjG5-wejt8XoIXO0zMtrAoj7Bi7UCWY3S6K4QDqBsB0a-8TrfCx6M3lOeu_Jr8A\" --redirect-url=\"https://remotedesktop.google.com/_/oauthredirect\" --name=$(hostname)" #@param {type:"string"}

#@markdown Enter a pin more or equal to 6 digits
Pin = 123456 #@param {type: "integer"}


class CRD:
    def __init__(self):
        os.system("apt update")
        self.installCRD()
        self.installDesktopEnvironment()
        self.installGoogleChorme()
        self.finish()

    @staticmethod
    def installCRD():
        print("Installing Chrome Remote Desktop")
        subprocess.run(['wget', 'https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['dpkg', '--install', 'chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def installDesktopEnvironment():
        print("Installing Desktop Environment")
        os.system("export DEBIAN_FRONTEND=noninteractive")
        os.system("apt install --assume-yes xfce4 desktop-base xfce4-terminal")
        os.system("bash -c 'echo \"exec /etc/X11/Xsession /usr/bin/xfce4-session\" > /etc/chrome-remote-desktop-session'")
        os.system("apt remove --assume-yes gnome-terminal")
        os.system("apt install --assume-yes xscreensaver")
        os.system("systemctl disable lightdm.service")

    @staticmethod
    def installGoogleChorme():
        print("Installing Google Chrome")
        subprocess.run(["wget", "https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(["dpkg", "--install", "google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)
        print ("installing wine")
        ! sudo apt install nautilus nano -y
        ! sudo apt-get install simplescreenrecorder
        ! sudo apt-get install sysmontask
        ! sudo apt-get install leafpad
        ! sudo pip3 install -U psutil

    @staticmethod
    def finish():
        print("Finalizing")
        os.system(f"adduser {username} chrome-remote-desktop")
        command = f"{CRP} --pin={Pin}"
        os.system(f"su - {username} -c '{command}'")
        os.system("service chrome-remote-desktop start")
        print("Finished Succesfully")


try:
    if username:
        if CRP == "":
            print("Please enter authcode from the given link")
        elif len(str(Pin)) < 6:
            print("Enter a pin more or equal to 6 digits")
        else:
            CRD()
except NameError as e:
    print("username variable not found")
    print("Create a User First")

Installing Chrome Remote Desktop
Installing Desktop Environment
Installing Google Chrome
installing wine
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  aspell aspell-en enchant gnome-terminal-data gstreamer1.0-gl
  gstreamer1.0-plugins-good gstreamer1.0-pulseaudio gstreamer1.0-x
  hunspell-en-us libaa1 libaspell15 libbrotli1 libdv4 libenchant1c2a
  libgraphene-1.0-0 libgstreamer-gl1.0-0 libgstreamer-plugins-good1.0-0
  libhunspell-1.6-0 libhyphen0 libjavascriptcoregtk-4.0-18 libshout3 libtag1v5
  libtag1v5-vanilla libv4l-0 libv4lconvert0 libwebkit2gtk-4.0-37 libwoff1
  libyelp0 yelp yelp-xsl
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  at-spi2-core bubblewrap gcr gnome-desktop3-data gnome-keyring
  gnome-keyring-pkcs11 gvfs-backends libdbusmenu-glib4 libdee-1.0-4 libexempi3
  libgail-3-0 libgcr-ui-3-1 libg

In [ ]:
#@title **Android Studio**
#@markdown  It takes 4-5 minutes for installation
%cd /home/user/Downloads
! wget https://redirector.gvt1.com/edgedl/android/studio/install/2021.1.1.22/android-studio-2021.1.1.22-cros.deb
! sudo apt-get install ./android-studio-2021.1.1.22-cros.deb
! clear
! echo "Android Studio Installed"

In [ ]:
%env USE_AUTH_EPHEM=0

#@title **Google Drive Mount**
#@markdown Google Drive can be used as Persistance HDD for files.<br>

#@markdown **Choose a method (GDFuse Recommended)**
mount_method = "GDFuse" #@param ["GDFuse", "Native"]


#@markdown **Options for GDFuse** <br>
#@markdown - Visit https://github.com/astrada/google-drive-ocamlfuse/wiki/Team-Drives
label = "default" #@param {type:"string"}
mount_team_drive = True #@param {type:"boolean"}
force_mount = False #@param {type:"boolean"}

import os
import subprocess


class Drive():
    creds = {}
    mountpoint = ""
    deps = False
    
    debug = False

    def __init__(self, mountpoint="/content/drives", debug=False):
        os.makedirs(mountpoint, exist_ok=True)
        self.mountpoint = mountpoint
        self.debug = debug

    def _mount_gdfuse(self, mount_dir):
        os.makedirs(mount_dir, exist_ok=True)

        subprocess.run(
            ['google-drive-ocamlfuse',
             '-o',
             'allow_other',
             '-label',
             label,
             mount_dir,
            ]
        )

        print(f"Drive Mounted at {mount_dir}. If you get input/output error, then `team_drive_id` might be wrong or not accessible.")

    def _unmount_gdfuse(self, mount_dir):
        subprocess.run(
            ['fusermount',
             '-u',
             mount_dir,
            ]
        )
        os.rmdir(mount_dir)

    def auth(self):
        from google.colab import auth
        from oauth2client.client import GoogleCredentials

        auth.authenticate_user()
        
        creds = GoogleCredentials.get_application_default()
        self.creds = {
            "id": creds.client_id,
            "secret": creds.client_secret
        }

    def gdfuse(self, label, mound_team_drive=False, force_mount=False):
        import getpass

        if not self.creds:
            self.auth()

        if not self.deps:
            print("Installing google-drive-ocamlfuse")
            subprocess.run(['apt', 'install', 'software-properties-common python-software-properties module-init-tools', '-y'])
            subprocess.run(['add-apt-repository', 'ppa:alessandro-strada/ppa', '-y'])
            subprocess.run(['apt', 'update'])
            subprocess.run(['apt', 'install', '--assume-yes', 'google-drive-ocamlfuse'])
            self.deps = True

        base_dir = '/root/.gdfuse'
        config_dir = f'{base_dir}/{label}'
        mount_dir = f"{self.mountpoint}/{label}"

        if force_mount and os.path.exists(mount_dir):
            self._unmount_gdfuse(mount_dir)
        elif os.path.exists(mount_dir):
            print("Drive already mounted")
            return

        if not os.path.exists(config_dir) or force_mount:
            print(f"Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id={self.creds['id']}&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force")
            vcode = getpass.getpass("Enter the Auth Code: ")

            subprocess.run(
              ['google-drive-ocamlfuse',
                '-headless',
                '-id',
                self.creds['id'],
                '-secret',
                self.creds['secret'],
                '-label',
                label,
              ],
                text=True,
                input=vcode
            )

        if mount_team_drive:
            team_drive_id = '0AIzwjZM2w4cuUk9PVA'

            subprocess.run(
                ['sed',
                 '-i',
                 f's/team_drive_id=.*$/team_drive_id={team_drive_id}/g',
                 f'{config_dir}/config'
                ]
            )
        else:
            subprocess.run(
                ['sed',
                 '-i',
                 f's/team_drive_id=.*$/team_drive_id=/g',
                 f'{config_dir}/config'
                ]
            )

        self._mount_gdfuse(mount_dir)


    def native(self):
        from google.colab import drive
        mount_dir = f"{self.mountpoint}/Native"
        drive.mount(mount_dir)

if 'drive' not in globals():
    try:
        drive = Drive(f"/home/{username}/drives")
    except NameError:
        drive = Drive('/content/drives')

if mount_method == "Native":
    drive.native()
if mount_method == "GDFuse":
    drive.gdfuse(label, mount_team_drive, force_mount)


In [ ]:
%cd /home/user/Downloads
! sudo apt-get install -y libgtk-3-0 libgbm-dev libatspi2.0-0 libatomic1 && wget -O 9hits-linux-x64.tar.xz https://www.dropbox.com/s/4edjcrqammmdmww/9hits-linux-x64.tar.xz?dl=1 && tar -xf 9hits-linux-x64.tar.xz